In [9]:
!pip install gdown

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [10]:
!gdown --id 1t6fwPgO5ml89zf4k5btrwRj9r83WzlcS

/usr/local/lib/python3.7/dist-packages/gdown/cli.py:131: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  category=FutureWarning,
Downloading...
From: https://drive.google.com/uc?id=1t6fwPgO5ml89zf4k5btrwRj9r83WzlcS
To: /content/combined-csv-files.csv
100% 483M/483M [00:10<00:00, 47.0MB/s]


In [5]:
# Activate Spark in our Colab notebook.
import os
# Find the latest version of spark 3.0  from http://www.apache.org/dist/spark/ and enter as the spark version
# For example: 'spark-3.2.2'
spark_version = 'spark-3.2.2'
# spark_version = 'spark-3.<enter version>'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop3.2.tgz
!tar xf $SPARK_VERSION-bin-hadoop3.2.tgz
!pip install -q findspark

# Set Environment Variables
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop3.2"

# Start a SparkSession
import findspark
findspark.init()

Get:1 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Hit:2 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:3 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Get:4 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Get:5 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [83.3 kB]
Get:6 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Ign:7 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Get:8 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease [1,581 B]
Hit:9 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Hit:10 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Hit:11 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease
Get:12 http://security.ubuntu.com/ubuntu bionic-security/universe amd64 Packages [1,546 kB]
Get:13

In [6]:
# Import packages
!pip install pyspark
from pyspark.sql import SparkSession
import time

# Create a SparkSession
spark=SparkSession.builder.appName("SparkSQL").getOrCreate()

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 281.3 MB 38 kB/s 
     |████████████████████████████████| 199 kB 61.5 MB/s 
  Created wheel for pyspark: filename=pyspark-3.3.0-py2.py3-none-any.whl size=281764026 sha256=b5ed442cfa16fd991b8be308c30ec4a647278297332af476cd87e51900fddf60
  Stored in directory: /root/.cache/pip/wheels/7a/8e/1b/f73a52650d2e5f337708d9f6a1750d451a7349a867f928b885
Successfully built pyspark


In [11]:
from pyspark import SparkFiles
df = spark.read.csv(SparkFiles.get('/content/combined-csv-files.csv'),sep=",", header=True,encoding='ISO-8859-1')
df.show(10)
print((df.count(),len(df.columns)))

+--------------------+---------+----+-------+----------------+-------------------+-----+---------+--------------------+--------------------+--------------------+
|           phone_url|     date|lang|country|          source|             domain|score|score_max|             extract|              author|             product|
+--------------------+---------+----+-------+----------------+-------------------+-----+---------+--------------------+--------------------+--------------------+
|/cellphones/samsu...| 5/2/2017|  en|     us|Verizon Wireless|verizonwireless.com|   10|       10|As a diehard Sams...|          CarolAnn35|   Samsung Galaxy S8|
|/cellphones/samsu...|4/28/2017|  en|     us|     Phone Arena|     phonearena.com|   10|       10|Love the phone. t...|           james0923|   Samsung Galaxy S8|
|/cellphones/samsu...| 5/4/2017|  en|     us|          Amazon|         amazon.com|    6|       10|Adequate feel. Ni...|            R. Craig|"Samsung Galaxy S...|
|/cellphones/samsu...| 5/2/2

In [ ]:
df.printSchema() 

root
 |-- phone_url: string (nullable = true)
 |-- date: string (nullable = true)
 |-- lang: string (nullable = true)
 |-- country: string (nullable = true)
 |-- source: string (nullable = true)
 |-- domain: string (nullable = true)
 |-- score: string (nullable = true)
 |-- score_max: string (nullable = true)
 |-- extract: string (nullable = true)
 |-- author: string (nullable = true)
 |-- product: string (nullable = true)



In [15]:
df = df.withColumn("score", df.score.cast('float'))
#df = df.withColumn('date', to_date(df.date,'MM-dd-yyyy'))
df.printSchema() 

root
 |-- phone_url: string (nullable = true)
 |-- date: string (nullable = true)
 |-- lang: string (nullable = true)
 |-- country: string (nullable = true)
 |-- source: string (nullable = true)
 |-- domain: string (nullable = true)
 |-- score: float (nullable = true)
 |-- score_max: string (nullable = true)
 |-- extract: string (nullable = true)
 |-- author: string (nullable = true)
 |-- product: string (nullable = true)



In [20]:
df = df.drop('extract', 'score_max') # dropping the colomns that we dont need
df = df.na.fill("anonymous",["author"]) # Filling null in author with anonymous
df = df.dropna(how = 'any') # Drop any null row
df.show()
print((df.count(),len(df.columns)))

+--------------------+---------+----+-------+----------------+-------------------+-----+--------------------+--------------------+
|           phone_url|     date|lang|country|          source|             domain|score|              author|             product|
+--------------------+---------+----+-------+----------------+-------------------+-----+--------------------+--------------------+
|/cellphones/samsu...| 5/2/2017|  en|     us|Verizon Wireless|verizonwireless.com|   10|          CarolAnn35|   Samsung Galaxy S8|
|/cellphones/samsu...|4/28/2017|  en|     us|     Phone Arena|     phonearena.com|   10|           james0923|   Samsung Galaxy S8|
|/cellphones/samsu...| 5/4/2017|  en|     us|          Amazon|         amazon.com|    6|            R. Craig|"Samsung Galaxy S...|
|/cellphones/samsu...| 5/2/2017|  en|     us|         Samsung|        samsung.com|  9.2|         Buster2020 |Samsung Galaxy S8...|
|/cellphones/samsu...|5/11/2017|  en|     us|Verizon Wireless|verizonwireless.com| 

In [21]:
from pyspark.sql.functions import udf
def ascii_ignore(i):
  return i.encode('ascii','ignore').decode('ascii')

ascii_udf = udf(ascii_ignore)
df = df.withColumn("source",ascii_udf("source"))
df = df.withColumn("author",ascii_udf("author"))
phones_df = df.withColumn("product",ascii_udf("product"))
phones_df.show()
print((phones_df.count(),len(phones_df.columns)))

+--------------------+---------+----+-------+----------------+-------------------+-----+--------------------+--------------------+
|           phone_url|     date|lang|country|          source|             domain|score|              author|             product|
+--------------------+---------+----+-------+----------------+-------------------+-----+--------------------+--------------------+
|/cellphones/samsu...| 5/2/2017|  en|     us|Verizon Wireless|verizonwireless.com|   10|          CarolAnn35|   Samsung Galaxy S8|
|/cellphones/samsu...|4/28/2017|  en|     us|     Phone Arena|     phonearena.com|   10|           james0923|   Samsung Galaxy S8|
|/cellphones/samsu...| 5/4/2017|  en|     us|          Amazon|         amazon.com|    6|            R. Craig|"Samsung Galaxy S...|
|/cellphones/samsu...| 5/2/2017|  en|     us|         Samsung|        samsung.com|  9.2|         Buster2020 |Samsung Galaxy S8...|
|/cellphones/samsu...|5/11/2017|  en|     us|Verizon Wireless|verizonwireless.com| 

In [ ]:
# Create the Great Expectations DataFrame
!pip install great_expectations
import great_expectations as ge
phones_df_ge = ge.dataset.SparkDFDataset(phones_df)

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
# Test that each column doesn't contain null values.
for column in ['phone_url', 'date', 'lang', 'country', 'source','domain','score','author','product']:
  print(phones_df_ge.expect_column_values_to_not_be_null(column=column)['success'])

True
True
True
True
True
True
True
True
True


In [ ]:
#querying on original dataframe to check time
phones_df.createOrReplaceTempView('phone')

start_time = time.time()
sql = ("""
select country,source,product, COUNT(source) as People
from phone
group by country,source,product
order by people desc
""")
spark.sql(sql).show()
print("--- %s seconds ---" % (time.time() - start_time))

+-------+----------+--------------------+------+
|country|    source|             product|People|
+-------+----------+--------------------+------+
|     in|    Amazon|Lenovo Vibe K4 No...|  5222|
|     in|    Amazon|Lenovo Vibe K4 No...|  4388|
|     in|    Amazon|OnePlus 3 (Graphi...|  4094|
|     in|    Amazon|OnePlus 3 (Soft G...|  3549|
|     in|    Amazon|Lenovo Vibe K5 (G...|  2530|
|     in|    Amazon|Lenovo Vibe K5 (G...|  2104|
|     in|    Amazon|Lenovo Used Lenov...|  1950|
|     in|    Amazon|OnePlus 3T (Gunme...|  1878|
|     in|  Snapdeal|Samsung Galaxy J3...|  1844|
|     us|   Samsung|Samsung Galaxy S7...|  1806|
|     it|    Amazon|"Huawei P8 Lite S...|  1729|
|     us|   Samsung|Samsung Galaxy S7...|  1724|
|     it|    Amazon|Huawei P9 Lite Sm...|  1689|
|     us|   Samsung|Samsung Galaxy S7...|  1602|
|     in|    Amazon|Apple iPhone 5s (...|  1601|
|     in|    Amazon|YU Yuphoria YU501...|  1587|
|     de|   Otto.de|Samsung Galaxy S7...|  1566|
|     be|KIESKEURIG|

In [ ]:
# Write out the data in parquet format
phones_df.write.parquet('parquet_phone',mode='overwrite')

# Read in our new parquet formatted data
p_df=spark.read.parquet('parquet_phone')

# We can convert the dataframe to a view.
p_df.createOrReplaceTempView('p_phone')

In [ ]:
from pyspark.sql.functions import *

#Rerun same query to check runtime in parquet format
start_time = time.time()
sql = ("""
select country,source,product, COUNT(source) as People
from p_phone
group by country,source,product
order by people desc
""")
spark.sql(sql).show()
print("--- %s seconds ---" % (time.time() - start_time))

+-------+----------+--------------------+------+
|country|    source|             product|People|
+-------+----------+--------------------+------+
|     in|    Amazon|Lenovo Vibe K4 No...|  5222|
|     in|    Amazon|Lenovo Vibe K4 No...|  4388|
|     in|    Amazon|OnePlus 3 (Graphi...|  4094|
|     in|    Amazon|OnePlus 3 (Soft G...|  3549|
|     in|    Amazon|Lenovo Vibe K5 (G...|  2530|
|     in|    Amazon|Lenovo Vibe K5 (G...|  2104|
|     in|    Amazon|Lenovo Used Lenov...|  1950|
|     in|    Amazon|OnePlus 3T (Gunme...|  1878|
|     in|  Snapdeal|Samsung Galaxy J3...|  1844|
|     us|   Samsung|Samsung Galaxy S7...|  1806|
|     it|    Amazon|"Huawei P8 Lite S...|  1729|
|     us|   Samsung|Samsung Galaxy S7...|  1724|
|     it|    Amazon|Huawei P9 Lite Sm...|  1689|
|     us|   Samsung|Samsung Galaxy S7...|  1602|
|     in|    Amazon|Apple iPhone 5s (...|  1601|
|     in|    Amazon|YU Yuphoria YU501...|  1587|
|     de|   Otto.de|Samsung Galaxy S7...|  1566|
|     be|KIESKEURIG|

In [24]:
# Four successful assertions that test for quality assurance during the data extraction or transformation process.

# Install pytest and pytest-sugar to make our output look nice.
!pip install -q pytest pytest-sugar


In [25]:
# Create a phones_data.py file and write the function to it. 
# This file will be stored in our pwd (/content/tests).
%%file phones_data.py

# Import findspark() and initialize. 
import findspark
findspark.init()

# Import other dependencies. 
from pyspark import SparkFiles
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("sparkBankData").getOrCreate()

# Create the import_data function. 
def clean_data():
  phones_df = spark.read.csv(SparkFiles.get('/content/combined-csv-files.csv'),sep=",", header=True, encoding='ISO-8859-1')
  phones_df = phones_df.drop("extract", "score_max") # dropping the colomns that we dont need
  phones_df = phones_df.na.fill("anonymous",["author"]) # Filling null in author with anonymous
  phones_df = phones_df.dropna(how = 'any') # Drop any null row
  phones_df = phones_df.where(phones_df.score != 'score') # filter out the word score
  return phones_df

def old_data():
  df = spark.read.csv(SparkFiles.get('/content/combined-csv-files.csv'),sep=",", header=True, encoding='ISO-8859-1')
  return df

Writing phones_data.py


In [41]:
# Create a test_phones_data.py file and write the test functions to it. 
# This file will be stored in our pwd (/content/tests).
# Start a SparkSession

%%file test_phones_data.py

# From the phones_data.py file and import the import_data function. 
from phones_data import clean_data
from phones_data import old_data

# Write the tests. 
def test_column_count():
  df = clean_data()
  assert len(df.columns) == 9

def test_columns_df():
  df = clean_data()
  assert df.schema.names == ['phone_url','date','lang','country','source','domain','score','author','product']

def test_rows_dropped():
  df = clean_data()
  old_df = old_data()
  assert len(df.columns) != len(old_df.columns)

def test_check_score_greater_than_zero():
  df = clean_data()
  values = df.select("score").distinct().collect()
  for value in values:
    assert not float(value.score) < 0


Overwriting test_phones_data.py


In [42]:
# Run the test_import_data.py file with pytest. 
!python -m pytest test_phones_data.py

Test session starts (platform: linux, Python 3.7.14, pytest 3.6.4, pytest-sugar 0.9.5)
rootdir: /content, inifile:
plugins: typeguard-2.7.1, sugar-0.9.5

 test_phones_data.py ✓✓✓✓                                        100% ██████████

Results (28.05s):
       4 passed
